In [ ]:
USE DATABASE DEV;
USE SCHEMA DATASCIENCE;

In [ ]:
CREATE OR REPLACE FILE FORMAT my_parquet_format
  TYPE = 'PARQUET';

In [ ]:
CREATE OR REPLACE STAGE bronze_xml
  URL = 's3://datascience-output-bucket/bronze/xml/'
  STORAGE_INTEGRATION = aws_snowflake_connect
  FILE_FORMAT = my_parquet_format;

In [ ]:
CREATE OR REPLACE TABLE customers (
  customer_id   INT,
  customer_name STRING,
  start_date TIMESTAMP_NTZ
);

CREATE OR REPLACE TABLE products (
  product_id    STRING,         
  product_name  STRING,
  start_date    TIMESTAMP_NTZ
);

CREATE OR REPLACE TABLE orders (
  order_id     STRING,        
  customer_id  INT,
  product_id   STRING,
  start_date   TIMESTAMP_NTZ
);

In [ ]:
from datetime import datetime
from snowflake.snowpark import Session

# Step 1: Folder path
folder_path = datetime.utcnow().strftime("%Y/%b/%d/").title()

# Step 2: Snowflake connection config
connection_parameters = {
    "account": "PL49411.ap-southeast-1",
    "user": "ZUBAIR2216",
    "password": "Datalabs@193001",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "DEV",
    "schema": "DATASCIENCE"
}

# Step 3: Create session
session = Session.builder.configs(connection_parameters).create()

# Step 4: COPY INTO customers
copy_customers = f"""
COPY INTO customers
FROM @bronze_xml/{folder_path}
FILE_FORMAT = (FORMAT_NAME = my_parquet_format)
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
PATTERN = '.*Customers_.*\\.parquet';
"""
session.sql(copy_customers).collect()
print("✅ Customers loaded")

# Step 5: COPY INTO products
copy_products = f"""
COPY INTO products
FROM @bronze_xml/{folder_path}
FILE_FORMAT = (FORMAT_NAME = my_parquet_format)
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
PATTERN = '.*Products_.*\\.parquet';
"""
session.sql(copy_products).collect()
print("✅ Products loaded")

# Step 6: COPY INTO orders
copy_orders = f"""
COPY INTO orders
FROM @bronze_xml/{folder_path}
FILE_FORMAT = (FORMAT_NAME = my_parquet_format)
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
PATTERN = '.*Orders_.*\\.parquet';
"""
session.sql(copy_orders).collect()
print("✅ Orders loaded")

# Step 7: Close session
session.close()
print("🔒 Session closed")

In [ ]:
print(folder_path)

In [ ]:
# Show top 10 rows from customers table
df = session.table("orders").limit(10)
df.show()

In [ ]:
select * from customers;

In [ ]:
select * from products;

In [ ]:
select * from orders;